In [1]:
# =-=-=-=-=-=-=-=-=-=-=
# Data Load & Tokenize
# =-=-=-=-=-=-=-=-=-=-= 

import pandas
import re
from nltk.tokenize import WhitespaceTokenizer

# LOAD
colnames = ['author', 'title', 'date' , 'length', 'text']
df = pandas.read_csv('../data/talks_3.csv', names=colnames)
talks = df.text.tolist()
authors = df.author.tolist()
dates = df.date.tolist()
years = [re.sub('[A-Za-z ]', '', item) for item in dates]
authordate = [author+" "+year for author, year in zip(authors, years)]

# TOKENIZE
tokenizer = WhitespaceTokenizer()
texts = []
for talk in talks:   
    raw = re.sub(r"[^\w\d'\s]+",'', talk).lower()
    tokens = tokenizer.tokenize(raw)
    texts.append(tokens)

In [2]:
# =-=-=-=-=-=-=-=-=-=-=
# Small Test Corpus
# =-=-=-=-=-=-=-=-=-=-= 

test = texts[0:5]

In [3]:
# =-=-=-=-=-=-=-=-=-=-=
# Function to collect word positions within a text (as a word list)
# =-=-=-=-=-=-=-=-=-=-= 

def word_positions(listname):
    from collections import defaultdict
    words_with_positions = defaultdict(list)
    for position, word in enumerate(listname):
        words_with_positions[word].append(float(position)/len(listname))
    return(words_with_positions)

# word_positions(texts[0]) # check output

In [6]:
# =-=-=-=-=-=-=-=-=-=-=
# Develop "super dictionary" of all the texts involved
# =-=-=-=-=-=-=-=-=-=-= 

super_dict = {}
for text in test:
    temp_dict = word_positions(text)
    for k, v in temp_dict.items():
        if super_dict.get(k) is None:
            super_dict[k] = []
        if v not in super_dict.get(k):
            # Possibly problematic for larger data sets
            super_dict[k] = super_dict[k] + v
            
# print(super_dict) # check output

In [7]:
super_dict["thank"]

[0.0,
 0.017037387600567912,
 0.9966871746332229,
 0.044716692189892805,
 0.4214395099540582,
 0.5565084226646249,
 0.7650842266462481,
 0.9191424196018376,
 0.9978560490045941,
 0.9994391475042064,
 0.373067191453472,
 0.5780151813325837,
 0.7880236154062412,
 0.8538093899353387,
 0.9994377284228282,
 0.9969465648854962]

In [36]:
import statistics

word = "global"

print("*" + word + "* occurs " + str(len(super_dict[word])) + 
      " times in the corpus, with an average position of " + 
      str(statistics.mean(super_dict[word])) + ".")

*global* occurs 7 times in the corpus, with an average position of 0.6342322621960647.


In [8]:
df = pandas.DataFrame()
df['word'] = super_dict.keys()
df['positions'] = super_dict.values()

In [17]:
df.head(10)

,word,positions
0,isn't,[0.20162647223780145]
1,meanwhile,[0.7053435114503817]
2,approtec,[0.2544278886702277]
3,waitress,[0.11878845243729295]
4,microsystems,[0.5344924284913067]
5,even,"[0.636447166921899, 0.10120888389091931, 0.284..."
6,more,"[0.3076194983435873, 0.3795551348793185, 0.391..."
7,typed,[0.2447166921898928]
8,nashville,"[0.06483672503549456, 0.06956933270231898, 0.1..."
9,shuttle,[0.5332312404287902]


In [18]:
len(df['positions'])

2329

In [40]:
# I need two additional columns: 
# one for mean, one for counting the number of items in a list

import numpy

df.apply(numpy.average(df['positions']))

TypeError: unsupported operand type(s) for /: 'list' and 'int'